# Fairmot Batch Inference in Amazon SageMaker

In this notebook, we will walk through the batch inference with the FairMOT model. Because [SageMaker Batch Transform only can run the batch transform job finishing within 600 seconds](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-batch-code.html), we use SageMaker processing to run the batch inference.

## 1. SageMaker Initialization 
First we upgrade SageMaker to the latest version. If your notebook is already using latest Sagemaker 2.x API, you may skip the next cell.

In [ ]:
! pip install --upgrade pip
! python3 -m pip install --upgrade sagemaker

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
print(f"SageMaker Execution Role:{role}")

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
aws_region = session.region_name
print(f"AWS region:{aws_region}")

container_name = "container-batch-inference"

## 2. Build and Push Amazon SageMaker Serving Container Images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to [Amazon ECR service](https://aws.amazon.com/ecr/).

### 2.1 Docker Environment Preparation

Because the volume size of container may exceed the available size in the root directory of the notebook instance, we need to put the directory of docker data into the ```/home/ec2-user/SageMaker/docker``` directory.

By default, the root directory of docker is set as ```/var/lib/docker/```. We need to change the directory of docker to ```/home/ec2-user/SageMaker/docker```. You can skip this step if you have done docker environment preparation in [`fairmot-training.ipynb`](fairmot-training.ipynb).

In [ ]:
!bash ./prepare-docker.sh

### 2.2 Build and Push FairMOT Serving Container Image

Use [`./container-batch-inference/build_tools/build_and_push.sh`](./container-batch-inference/build_tools/build_and_push.sh) script to build and push the [FairMOT](https://github.com/ifzhang/FairMOT) <b>batch inference</b> container image to Amazon ECR. 

In [ ]:
!cat ./{container_name}/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./{container_name}/build_tools/build_and_push.sh {aws_region}

In [ ]:
fairmot_image = f"{account}.dkr.ecr.{aws_region}.amazonaws.com/fairmot-sagemaker:pytorch1.8-batch-inference"

## 3. Run Batch Inference

Create an instance of SageMaker Processing with the built container above.

In [ ]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(
                image_uri=fairmot_image,
                role=role,
                instance_count=1,
                instance_type='ml.p3.2xlarge',
                command=['python3'])

Set the S3 URIs for the test data, the trained model and the result data.

In [ ]:
bucket_name = "fairmot-sg-east-1" # your-s3-bucket-name

# Restore the s3 uri of the trained model
%store -r s3_model_uri
#s3_model_uri="s3://{bucket_name}/{predix_model}/model.tar.gz" you can define the s3 uri of the trained model manually.

s3_output=f"s3://{bucket_name}/fairmot/sagemaker/output/batch-inference"

We use [MOT16-03](https://motchallenge.net/sequenceVideos/MOT16-04-raw.webm) from MOT challenge to test the batch inference. First, we download the video to the notebook instance and then upload it to the defined s3 bucket.

In [ ]:
!wget https://raw.githubusercontent.com/ifzhang/FairMOT/master/videos/MOT16-03.mp4
!aws s3 cp MOT16-03.mp4 {s3_input}/MOT16-03.mp4
!rm MOT16-03.mp4

Next, we run the sagemaker processing job.

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
script_processor.run(
    code='./container-batch-inference/processing.py',
    inputs=[
        ProcessingInput(source=s3_input, destination="/opt/ml/processing/input"),
        ProcessingInput(source=s3_model_uri, destination="/opt/ml/processing/model"),
    ], 
    outputs=[
        ProcessingOutput(source='/opt/ml/processing/output', destination=s3_output),
    ]
)

You can check the result saved in `s3://{bucket-name}/fairmot/sagemaker/output/batch-inference`.